In [ ]:
!pip install peft
!pip install huggingface_hub

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from huggingface_hub import HfApi, HfFolder, Repository

# Your Hugging Face token
huggingface_token = "insert hugging face token"
HfFolder.save_token(huggingface_token)

# Base model name
model_name = "insert base model"

# Path to the private adapter model
adapter_model_path = "path the adapter model"

# Directory to save the merged model and tokenizer
save_directory = "./merged_model"

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the private adapter model with the token
ft_model = PeftModel.from_pretrained(base_model, adapter_model_path, use_auth_token=huggingface_token)

# Merge the adapter with the base model and unload
model2 = ft_model.merge_and_unload()

# Save the merged model and tokenizer
model2.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Push the model to Hugging Face
repository_id = "insert path"
api = HfApi()
api.create_repo(repo_id=repository_id, private=True, exist_ok=True)
repository = Repository(local_dir=save_directory, clone_from=repository_id)# This Python 3 environment comes with many helpful analytics libraries installed
